In [16]:
import os
import hashlib
from collections import defaultdict
import json
import time

In [17]:
def get_hash2files(rootdir, n=None, verbose=False):
    hash2files = defaultdict(list)
    total_files = 0
    for dirpath, dirnames, filenames in os.walk(rootdir):
        for fname in filenames:
            fullpath = os.path.join(dirpath, fname)
            fh=file(fullpath,'r')
            md5=hashlib.md5()
            md5.update(fh.read(100<<20))
            hash2files[md5.hexdigest()].append(fullpath)
            total_files += 1
            if verbose:
                print fullpath
            if total_files % 4000 == 0:
                print "files done: %d" % total_files
            if n and len(hash2files)>=n:
                return hash2files, total_files
    return hash2files, total_files

In [19]:
t0 = time.time()
hash2files, total_files = get_hash2files('/Volumes/G-DRIVE USB/SeagateDrive')
savefile = file('hash2files.json','w')
json.dump(hash2files, savefile)
savefile.close()
sec = time.time()-t0
min= sec/60.0
hr = min/60.0
print "When through %d files in %.2f sec = %.2f min = %.2f hr" % (total_files, sec, min, hr)

files done: 4000
files done: 8000
files done: 12000
files done: 16000
files done: 20000
files done: 24000
files done: 28000
files done: 32000
files done: 36000
files done: 40000
files done: 44000
files done: 48000
files done: 52000
files done: 56000
files done: 60000
files done: 64000
files done: 68000
files done: 72000
files done: 76000
files done: 80000
files done: 84000
files done: 88000
files done: 92000
files done: 96000
files done: 100000
files done: 104000
files done: 108000
files done: 112000
files done: 116000
files done: 120000
files done: 124000
files done: 128000
files done: 132000
files done: 136000
files done: 140000
files done: 144000
files done: 148000
files done: 152000
files done: 156000
files done: 160000
files done: 164000
files done: 168000
files done: 172000
files done: 176000
files done: 180000
files done: 184000
files done: 188000
files done: 192000
files done: 196000
files done: 200000
files done: 204000
files done: 208000
files done: 212000
files done: 216000


In [20]:
hash2files = json.load(file('hash2files.json','r'))

In [55]:
def dedup(hash2files):
    dups = {}
    longest = None
    for md5sum, flist in hash2files.iteritems():
        if len(flist)>0:
            dups[md5sum] = [fname for fname in flist]
            if longest:
                if len(flist)> len(longest):
                    longest = [fname for fname in flist]
            else:
                longest = [fname for fname in flist]
    return dups, longest

In [58]:
longest[0:3]


[u'/Volumes/G-DRIVE USB/SeagateDrive/Documents/Documents/Web/joomla/administrator/cache/index.html',
 u'/Volumes/G-DRIVE USB/SeagateDrive/Documents/Documents/Web/joomla/administrator/components/index.html',
 u'/Volumes/G-DRIVE USB/SeagateDrive/Documents/Documents/Web/joomla/administrator/components/com_admin/index.html']

In [59]:
file(longest[0],'r').read()

'<!DOCTYPE html><title></title>\n'

In [57]:
dups, longest = dedup(hash2files)

In [25]:
def dedup_filename(hash2files):
    base_fname_to_paths = defaultdict(list)
    for flist in hash2files.values():
        for fname in flist:
            parentdir, basefname = os.path.split(fname)
            base_fname_to_paths[basefname].append(parentdir)
    to_delete = []
    for ky,lst in base_fname_to_paths.iteritems():
        if len(lst)==1:
            to_delete.append(ky)
    for ky in to_delete:
        del base_fname_to_paths[ky]
    return base_fname_to_paths

In [26]:
bp = dedup_filename(hash2files)

In [27]:
len(bp)


58314

In [38]:
fname, dirlist = bp.items()[0]

fname1 = os.path.join(dirlist[0], fname)
fname2 = os.path.join(dirlist[1], fname)
print fname1
print fname2

/Volumes/G-DRIVE USB/SeagateDrive/Pictures/Pictures/Wedding/5 Bride and Groom Portraits/IMG_9622.JPG
/Volumes/G-DRIVE USB/SeagateDrive/Pictures/Wedding/5 Bride and Groom Portraits/IMG_9622.JPG


In [40]:

fh = file(fname2,'r')
md5=hashlib.md5()
md5.update(fh.read(100<<20))
md5.hexdigest()
 

'cecd1aabeba43864da79c9ae78da8bf6'

In [41]:
hash2files['cecd1aabeba43864da79c9ae78da8bf6']

[u'/Volumes/G-DRIVE USB/SeagateDrive/Pictures/Pictures/Wedding/5 Bride and Groom Portraits/IMG_9622.JPG',
 u'/Volumes/G-DRIVE USB/SeagateDrive/Pictures/Wedding/5 Bride and Groom Portraits/IMG_9622.JPG']